In [3]:
import json
import re
import os
from nltk.corpus import stopwords
import nltk
import spacy
from heapq import nlargest
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
file_root = "./articles"
filenames = os.listdir(file_root)

In [6]:
regexp = r"([a-z0-9]*.[json])"
data = []
data_clean = []
tags = []
filenames_echan = filenames
#filenames_echan = './articles/0a0a9b07b057c4f5f9364228343a7427.json'

for filename in filenames_echan[:]:
    if filename[-10:]!='-info.json':
        with open(file_root+'/'+filename, 'r', encoding='utf-8') as f:
            article_fields = json.load(f)
            text = ''
            text_tag = ''
            text_clean = ''
            for key, value in article_fields.items():
                if key == 'title':
                    text = text +(' '+value['fr'])*3 #On met un poids de 3 au titre
                    text_clean = text_clean + (' '+value['fr']) + '\n\n'
                if key == 'chapters':
                    first = 1
                    for field in article_fields['chapters']:
                        for key, value in field.items():
                            if key == 'text':
                                if first == 1:
                                    first = 0
                                    text = text +(' '+ field['text']['fr'])*2 #On met un poids de 2 au premier paragraphe
                                    text_clean = text_clean +(' '+ field['text']['fr']) + '\n\n'
                                else:
                                    text = text +' '+ field['text']['fr']
                                    text_clean = text_clean +(' '+ field['text']['fr']) + '\n\n'
                if key == 'channel':
                    if article_fields['channel'] != {}:
                        text_tag = text_tag +article_fields['channel']['fr']
            data.append(text)
            tags.append(text_tag)
            data_clean.append(text_clean)
            if len(data) >= 50000: #On utilise seulement les 25000 premiers articles
                break


In [7]:
#Nettoyage du text (elever !, ? \n ...)
supprimer = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\n)|(\$)|(href)|(https)|(\%)|(—)")
remplacer_par_espace = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)|(')|(<[a-z]>)|(</[a-z]>)|»|«|(’)|(<p>)|(</p>)|(<i>)|(</em>)"\
                       "|(<em>)|(</i>)|(<b>)|(</b>)|(<br>)|(</br>)|(<strong>)|(</strong>)|(\*)|(&nbsp)|(&)|(<)|(>)|(=)")

def nettoyage(data):
    data = [supprimer.sub("", line.lower()) for line in data]
    data = [remplacer_par_espace.sub(" ", line) for line in data]
    return data

data_net = nettoyage(data)


In [8]:

def delete_words(documents):
    word_list = stopwords.words('french')
    word_list.extend(['cette','a','donc','car','ni','or','lors','tel','telle','e','h','no','st', 'aussi','très',\
                       'alors','toutefois','finalement','tant','outre','puis','ça','si','enfin','cependant','également'\
                     'mais','contrepartie','néanmoins', 'pourtant', 'or', 'malgré', 'que','puisque','puisqu','notamment'\
                     'tellement','bien'])
    output = []
    for line in documents:
        output.append(
            ' '.join([word for word in line.split() 
                      if word not in word_list]))
    return output

data_stw = delete_words(data_net)

In [9]:
def do_count(articles):
    '''Compter les occurences de chaque mot'''
    final_count=[]
    for article in articles:
        token_words = nltk.word_tokenize(article)
        final_count.append(nltk.FreqDist(token_words))
    return final_count

data_freq_dict = do_count(data_stw)

In [10]:
def dict_to_lists(articles):
    word_list = []
    word_freq = []
    for article in articles:
        word_list.append(list(article.keys()))
        word_freq.append(list(article.values()))
    return word_list, word_freq

word_list, word_freq = dict_to_lists(data_freq_dict)

In [11]:
for i in range(len(data)):
    with open('processed/article_'+str(i)+'.pickle', 'wb') as f:
        pickle.dump([word_list[i],word_freq[i]], f)

In [12]:
for i in range(len(data)):
    with open('printable/article_'+str(i)+'.pickle', 'wb') as f:
        pickle.dump([data_clean[i],tags[i]], f)